<a href="https://colab.research.google.com/github/ChaitanyaDeshmukh99/mai_project1_optimization/blob/main/copy_of_sandbox_2(Final).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False


In [ ]:
if IN_COLAB:
    !git clone https://github.com/LeonLaumeyer/mai_project1_optimization.git

fatal: destination path 'mai_project1_optimization' already exists and is not an empty directory.


In [ ]:
if IN_COLAB:
    !pip3 install -r mai_project1_optimization/requirements.txt
    !pip3 install optuna
    !pip3 install transformers

In [ ]:
!pip3 install accelerate


In [ ]:
!pip3 install torchmetrics

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import models
from torchvision.models import *
from plotly import express as px
from collections import Counter
import numpy as np
import random

if(IN_COLAB):
    from mai_project1_optimization.modules.dataset import IntelImageClassificationDataset
    from mai_project1_optimization.modules.utility import NotebookPlotter, InferenceSession, Evaluator, ISO_time, apply_pruning
    from mai_project1_optimization.modules.trainer import Trainer
    from mai_project1_optimization.modules.optuna_optimizer import OptunaTuner
    from mai_project1_optimization.modules.optuna_monashara import run_optuna
else:
    from modules.dataset import IntelImageClassificationDataset
    from modules.utility import NotebookPlotter, InferenceSession, Evaluator, ISO_time, apply_pruning
    from modules.trainer import Trainer
    from modules.optuna_optimizer import OptunaTuner
    from modules.optuna_monashara import run_optuna

torch.manual_seed(1)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

def set_seed(seed=1):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True  # for reproducibility
    torch.backends.cudnn.benchmark = False

# Removed support for Tensor Units
# torch.backends.cudnn.allow_tf32 = True
# torch.backends.cuda.matmul.allow_tf32 = True

set_seed(1)

cpu


https://www.kaggle.com/datasets/puneet6060/intel-image-classification

In [ ]:
# labels, values = zip(*Counter([item[1] for item in dataset.train_dataset]).items())
# fig = px.bar(x=labels, y=values, labels={'x': 'Categories', 'y': 'Counts'}, title='Distribution of Classes')
# fig.show()

| n | label |
| --- | --- |
| 0 | buildings |
| 1 | forest |
| 2 | glacier |
| 3 | mountain |
| 4 | sea |
| 5 | street |

NotebookPlotter.plot_dataset_item_interactive(dataset.train_dataset)

In [ ]:
choice = 1 # 1,2,3
freezeLayer = False
prune_model = False
USE_OPTUNA = False
OPTUNA_MO = False

if choice != 5:
    dataset = IntelImageClassificationDataset(resize=(150,150))
else:
    dataset = IntelImageClassificationDataset(resize=(384,384))

# 80% train, 20% validation for training Optuna
train_size = int(0.8 * len(dataset.train_dataset))
val_size = len(dataset.train_dataset) - train_size
train_subset, val_subset = random_split(dataset.train_dataset, [train_size, val_size], generator=torch.Generator().manual_seed(1))

def build_model():

  # SqueezeNet 1.1
  if choice == 1:
      model = models.squeezenet1_1(weights=SqueezeNet1_1_Weights.DEFAULT)
      num_features = model.classifier[1].in_channels
      kernel_size = model.classifier[1].kernel_size
      if(freezeLayer):
          for param in model.parameters():
              param.requires_grad = False
      model.classifier[1] = nn.Conv2d(num_features, 6, kernel_size)


  # MobileNetV2
  elif choice == 2:
      model = models.mobilenet_v2(weights=MobileNet_V2_Weights.DEFAULT)
      num_features = model.classifier[1].in_features
      if(freezeLayer):
          for param in model.parameters():
              param.requires_grad = False
      model.classifier[1] = nn.Linear(num_features, 6)

  # MobileNetV3 Small
  elif choice == 3:
      model = models.mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.DEFAULT)
      num_features = model.classifier[3].in_features
      if(freezeLayer):
          for param in model.parameters():
              param.requires_grad = False
      model.classifier[3] = nn.Linear(num_features, 6)

  # MobileNetV3 Large
  elif choice == 4:
      model = models.mobilenet_v3_large(weights=MobileNet_V3_Large_Weights.DEFAULT)
      num_features = model.classifier[3].in_features
      if(freezeLayer):
          for param in model.parameters():
              param.requires_grad = False
      model.classifier[3] = nn.Linear(num_features, 6)

  # VisionTransformer Base 16
  elif choice == 5:
      model = models.vit_b_16(weights=ViT_B_16_Weights.IMAGENET1K_SWAG_E2E_V1)
      num_features = model.heads[0].in_features
      if(freezeLayer):
          for param in model.parameters():
              param.requires_grad = False
      model.heads[0] = nn.Linear(num_features, 6)

  if prune_model:
    model = apply_pruning(model, amount=0.3)

  return model

if USE_OPTUNA:
    tuner = OptunaTuner(
        model_fn=build_model,
        train_dataset=train_subset,
        val_dataset=val_subset,
        device=DEVICE
    )
    study = tuner.run(n_trials=10, seed=1)
    print("Best trial parameters:", study.best_trial.params)

    best_params = study.best_trial.params
    model = build_model()
    dataloader = DataLoader(train_subset, batch_size=best_params["batch_size"], shuffle=True)
    trainer = Trainer(model=model, lr=best_params["lr"], device=DEVICE)
    epochs = best_params["epochs"]

elif OPTUNA_MO:

    model = build_model()

    best_params, best_model_state, study = run_optuna(
        model=model,
        train_subset=train_subset,
        val_subset=val_subset,
        TrainerClass=Trainer,
        n_trials=10,
        seed=1
    )

    print("▶ Per-epoch validation accuracy (best trial):")
    best_trial = study.best_trial
    for epoch, acc in sorted(best_trial.intermediate_values.items()):
        print(f"   Epoch {epoch:2d}: {acc * 100:.2f}%")

    print(f"\n▶ Best hyperparameters: {best_params}")
    print(f"▶ Best overall accuracy: {study.best_value * 100:.2f}%")

    model.load_state_dict(best_model_state)
    dataloader = DataLoader(
        dataset.train_dataset,
        batch_size=best_params["BS_SUGGEST"],
        shuffle=True
    )
    trainer = Trainer(
        model=model,
        lr=best_params["LR_SUGGEST"],
        device=DEVICE
    )
    epochs = best_params["EPOCHS"]

else:
    model = build_model()
    dataloader = DataLoader(dataset.train_dataset, batch_size=32, shuffle=True)
    trainer = Trainer(model=model, lr=8.841926348917726e-05, device=DEVICE)
    epochs = 25

Downloading: "https://download.pytorch.org/models/squeezenet1_1-b8a52dc0.pth" to /root/.cache/torch/hub/checkpoints/squeezenet1_1-b8a52dc0.pth
100%|██████████| 4.73M/4.73M [00:00<00:00, 57.6MB/s]


In [ ]:
# model.load_state_dict(torch.load(f"checkpoints/.pt"))
trainer.train(dataloader, epochs=epochs, silent=False)

/usr/local/lib/python3.11/dist-packages/torch/autograd/profiler.py:266: UserWarning: CUDA is not available, disabling CUDA profiling
  warn("CUDA is not available, disabling CUDA profiling")


  0%|          | 0/25 [00:00<?, ?it/s]

In [ ]:
session = InferenceSession(model)
output = session(torch.stack(tuple(item[0] for item in dataset.test_dataset)))
Evaluator.acc(output, torch.tensor(tuple(item[1] for item in dataset.test_dataset))).item()

In [ ]:
# torch.save(model.state_dict(), f"checkpoints/{model.__class__.__name__}.pt")

# Method No 1
#1) Efficient GPU memory management, including releasing unused memory



In [ ]:
# Method No 1
#1) Efficient GPU memory management, including releasing unused memory


def release_gpu_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

        import gc
        gc.collect()
        print("GPU memory released.")
    else:
        print("No GPU available to release memory.")

release_gpu_memory()


# Method No 2
#2) Executing asynchronous GPU operations and using non-blocking memory transfers

In [ ]:
# Method No 2
#2) Executing asynchronous GPU operations and using non-blocking memory transfers
dataloader = DataLoader(
    dataset.train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)
trainer = Trainer(model=model, lr=8.841926348917726e-05, device=DEVICE)
epochs = 25


trainer.train(dataloader, epochs=epochs, silent=False)
session = InferenceSession(model)

test_inputs = torch.stack(tuple(item[0] for item in dataset.test_dataset)).to(DEVICE, non_blocking=True)
test_labels = torch.tensor(tuple(item[1] for item in dataset.test_dataset)).to(DEVICE, non_blocking=True)

output = session(test_inputs)

Evaluator.acc(output, test_labels).item()

In [ ]:
# Profiler Code
import torch
import torch.profiler


sample_input, _ = next(iter(dataloader))
sample_input = sample_input.to(DEVICE)


with torch.profiler.profile(
    activities=[
        torch.profiler.ProfilerActivity.CPU,
        torch.profiler.ProfilerActivity.CUDA
    ],
    record_shapes=True,

    with_stack=True
) as prof:
    model(sample_input)

prof.export_chrome_trace("model_profile.json")

In [ ]:
from google.colab import files
files.download("model_profile.json")
